In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen, urlretrieve, quote
from urllib.parse import urljoin
import pandas as pd
import csv
from tqdm import tqdm

# Function

In [2]:
base_url = "https://id.wikipedia.org"
url1 = "https://id.wikipedia.org/wiki/Kategori:Daftar_bupati"
url2 = "https://id.wikipedia.org/w/index.php?title=Kategori:Daftar_bupati&pagefrom=Malinau%0ADaftar+Bupati+Malinau#mw-pages"


In [3]:
def get_url(url):
    headers = {
         'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36'
     }
    r = requests.get(url, headers=headers)  # Using the custom headers we defined above
    soup = BeautifulSoup(r.content, 'lxml') 
    return soup

# Get Data

In [4]:
list_link = []
list_text = []

for url in [url1, url2]:
    soup = get_url(url)
    spans = soup.find_all('div', {"class":"mw-category"})
    for x in spans[0].find_all('a', href=True):
        list_text.append(x.string)
        list_link.append(base_url+x['href'])

In [5]:
len(list_link), len(list_text)

(339, 339)

In [820]:
no_idx = 338
link_idx = list_link[no_idx]
print(link_idx)
daerah_sumber = link_idx.split("/")[-1]
df = pd.read_html(link_idx)[0].T
df = df[1:]
df["daerah_sumber"] = daerah_sumber
df

https://id.wikipedia.org/wiki/Daftar_Bupati_Yalimo


,0,1,2,3,4,5,daerah_sumber
Bupati Yalimo Ribka Haluk.1,NaN,PetahanaRibka Haluk (plt)sejak 26 Agustus 2021,5 tahun (definitif),21 Juni 2008,Ellia Ibrahim Loupatty,Situs Resmi Kabupaten Yalimo,Daftar_Bupati_Yalimo


In [817]:
df = df.rename(columns={1:"nama", 2:"alamat"})
df = df[["nama", "alamat", "daerah_sumber"]].reset_index(drop=True)
df

,nama,alamat,daerah_sumber
0,PetahanaRibka Haluk (plt)sejak 26 Agustus 2021,5 tahun (definitif),Daftar_Bupati_Yalimo


In [818]:
df.to_csv("./Bupati/{}.csv".format(daerah_sumber), index=False)

In [800]:
df = pd.read_html(link_idx)[0]
df

,No.,Foto,Nama Bupati,Awal Menjabat,Akhir Menjabat,Wakil Bupati,Keterangan,Ref.
0,1,NaN,"Syarifudin Safaa, S.H., M.M.",19 Januari 2004,19 Januari 2006,NaN,Penjabat bupati,NaN
1,2,NaN,"H. L.M. Mahufi Madra, S.H., M.H.",19 Januari 2006,28 Juni 2006,NaN,Penjabat bupati,NaN
2,3,NaN,Ir. Hugua,28 Juni 2006,28 Juni 2011,"Ediarto Rusmin, B.A.E.",Periode pertama,NaN
3,3,NaN,Ir. Hugua,28 Juni 2011,28 Juni 2016,"H. Arhawi, S.E.",Periode kedua,NaN
4,4,NaN,"H. Arhawi, S.E.",28 Juni 2016,sekarang,"Ilmiati Daud, S.E., M.Si.",Bupati definitif,[2]


In [801]:
# df = df.droplevel(0, axis=1)

In [802]:
df = df.iloc[[-1]]
df

,No.,Foto,Nama Bupati,Awal Menjabat,Akhir Menjabat,Wakil Bupati,Keterangan,Ref.
4,4,NaN,"H. Arhawi, S.E.",28 Juni 2016,sekarang,"Ilmiati Daud, S.E., M.Si.",Bupati definitif,[2]


In [805]:
df = df.rename(columns={"Nama Bupati":"nama"})
df["alamat"] = "No Data"
df["daerah_sumber"] = daerah_sumber
df = df[["nama", "alamat", "daerah_sumber"]].reset_index(drop=True)
df

,nama,alamat,daerah_sumber
0,"H. Arhawi, S.E.",No Data,Daftar_Bupati_Wakatobi


In [806]:
df.to_csv("./Bupati/{}.csv".format(daerah_sumber), index=False)

In [15]:
list_success = []
list_failed = []
for idx in tqdm(range(len(list_link))):
    try:
        df = pd.read_html(list_link[idx])[0].T
#         new_header = df.iloc[0]
        df = df[1:]
#         df.columns = new_header
        df["Daerah Sumber"] = list_text[idx]
        df = df.loc[:,~df.columns.duplicated()]
        list_success.append(df)
    except:
        list_failed.append(list_link[idx])

100%|████████████████████████████████████████| 339/339 [01:01<00:00,  5.55it/s]


In [14]:
df = pd.concat(list_success)
df

,0,1,2,3,4,5,6,Daerah Sumber,7,8,...,26,27,28,29,30,31,32,33,34,35
Bupati Aceh Barat,NaN,PetahanaRamli MSsejak 10 Oktober 2017,Kediaman,Masa jabatan,Dibentuk,Pejabat pertama,Situs web,Daftar Bupati Aceh Barat,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bupati Aceh Barat.1,NaN,PetahanaRamli MSsejak 10 Oktober 2017,Pendopo Jalan Pocut Baren,5 tahun (definitif),1945,H. Ibnu Sa'dan,Situs Resmi Kabupaten Aceh Barat,Daftar Bupati Aceh Barat,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bupati Aceh Barat Daya,NaN,PetahanaAkmal Ibrahimsejak 13 Agustus 2017,Kediaman,Masa jabatan,Dibentuk,Pejabat pertama,Situs web,Daftar Bupati Aceh Barat Daya,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bupati Aceh Barat Daya.1,NaN,PetahanaAkmal Ibrahimsejak 13 Agustus 2017,Pendopo Bupati Aceh Barat Daya,5 tahun (definitif),2006,Baharuddin,acehbaratdayakab.go.id,Daftar Bupati Aceh Barat Daya,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bupati Aceh Jaya,NaN,PetahanaTengku Irfan TBsejak 18 Juli 2017,Masa jabatan,Dibentuk,Pejabat pertama,Situs web,NaN,Daftar Bupati Aceh Jaya,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Bupati Way Kanan.1,Lambang Kabupaten Way Kanan,PetahanaRaden Adipati Suryasejak 17 Februari 2016,Pendapa Kabupaten Way Kanan,5 tahun,1986,Muhtar Lutfi,www.waykanankab.go.id,Daftar Bupati Way Kanan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bupati Yahukimo,NaN,PetahanaDidimus Yahulisejak 4 Mei 2021,Masa jabatan,Situs web,NaN,NaN,NaN,Daftar Bupati Yahukimo,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bupati Yahukimo.1,NaN,PetahanaDidimus Yahulisejak 4 Mei 2021,5 tahun (definitif),Situs Resmi Kabupaten Yahukimo,NaN,NaN,NaN,Daftar Bupati Yahukimo,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bupati Yalimo Ribka Haluk,NaN,PetahanaRibka Haluk (plt)sejak 26 Agustus 2021,Masa jabatan,Dibentuk,Pejabat pertama,Situs web,NaN,Daftar Bupati Yalimo,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
for idx in range(len(list_success)):
    display(list_success[idx])

,0,1,2,3,4,5,6,Daerah Sumber
Bupati Aceh Barat.1,NaN,PetahanaRamli MSsejak 10 Oktober 2017,Pendopo Jalan Pocut Baren,5 tahun (definitif),1945,H. Ibnu Sa'dan,Situs Resmi Kabupaten Aceh Barat,Daftar Bupati Aceh Barat


,0,1,2,3,4,5,6,Daerah Sumber
Bupati Aceh Barat Daya.1,NaN,PetahanaAkmal Ibrahimsejak 13 Agustus 2017,Pendopo Bupati Aceh Barat Daya,5 tahun (definitif),2006,Baharuddin,acehbaratdayakab.go.id,Daftar Bupati Aceh Barat Daya


,0,1,2,3,4,5,Daerah Sumber
Bupati Aceh Jaya.1,NaN,PetahanaTengku Irfan TBsejak 18 Juli 2017,5 tahun,2002,Zulfian Ahmad,Situs Resmi Kabupaten Aceh Jaya,Daftar Bupati Aceh Jaya


,0,1,2,3,4,5,Daerah Sumber
Bupati Aceh Selatan.1,NaN,PetahanaTgk. Amransejak 3 Desember 2019,5 tahun,1945,Muhammad Sahim Hasjmy,Situs Resmi Kabupaten Aceh Selatan,Daftar Bupati Aceh Selatan


,0,1,2,3,4,Daerah Sumber
Bupati Aceh Singkil.1,NaN,PetahanaDulmusridsejak 21 Juli 2017,5 tahun,Makmur Syahputra Bancin,Situs Resmi Kabupaten Aceh Singkil,Daftar Bupati Aceh Singkil


...

,0,1,2,3,4,Daerah Sumber
Foto,NaN,NaN,NaN,NaN,NaN,Daftar Bupati Wakatobi
Nama Bupati,"Syarifudin Safaa, S.H., M.M.","H. L.M. Mahufi Madra, S.H., M.H.",Ir. Hugua,Ir. Hugua,"H. Arhawi, S.E.",Daftar Bupati Wakatobi
Awal Menjabat,19 Januari 2004,19 Januari 2006,28 Juni 2006,28 Juni 2011,28 Juni 2016,Daftar Bupati Wakatobi
Akhir Menjabat,19 Januari 2006,28 Juni 2006,28 Juni 2011,28 Juni 2016,sekarang,Daftar Bupati Wakatobi
Wakil Bupati,NaN,NaN,"Ediarto Rusmin, B.A.E.","H. Arhawi, S.E.","Ilmiati Daud, S.E., M.Si.",Daftar Bupati Wakatobi
Keterangan,Penjabat bupati,Penjabat bupati,Periode pertama,Periode kedua,Bupati definitif,Daftar Bupati Wakatobi
Ref.,NaN,NaN,NaN,NaN,[2],Daftar Bupati Wakatobi


,0,1,2,3,4,Daerah Sumber
Bupati Waropen.1,NaN,PetahanaYermias Bisaisejak 15 Maret 2021,5 tahun,25 Oktober 2002,www.waropenkab.go.id,Daftar Bupati Waropen


,0,1,2,3,4,5,6,Daerah Sumber
Bupati Way Kanan.1,Lambang Kabupaten Way Kanan,PetahanaRaden Adipati Suryasejak 17 Februari 2016,Pendapa Kabupaten Way Kanan,5 tahun,1986,Muhtar Lutfi,www.waykanankab.go.id,Daftar Bupati Way Kanan


,0,1,2,3,Daerah Sumber
Bupati Yahukimo.1,NaN,PetahanaDidimus Yahulisejak 4 Mei 2021,5 tahun (definitif),Situs Resmi Kabupaten Yahukimo,Daftar Bupati Yahukimo


,0,1,2,3,4,5,Daerah Sumber
Bupati Yalimo Ribka Haluk.1,NaN,PetahanaRibka Haluk (plt)sejak 26 Agustus 2021,5 tahun (definitif),21 Juni 2008,Ellia Ibrahim Loupatty,Situs Resmi Kabupaten Yalimo,Daftar Bupati Yalimo


In [8]:
list_success

[Bupati Aceh Barat    NaN  PetahanaRamli MSsejak 10 Oktober 2017  \
 Bupati Aceh Barat.1  NaN  PetahanaRamli MSsejak 10 Oktober 2017   
 
 Bupati Aceh Barat                     Kediaman         Masa jabatan Dibentuk  \
 Bupati Aceh Barat.1  Pendopo Jalan Pocut Baren  5 tahun (definitif)     1945   
 
 Bupati Aceh Barat   Pejabat pertama                         Situs web  \
 Bupati Aceh Barat.1  H. Ibnu Sa'dan  Situs Resmi Kabupaten Aceh Barat   
 
 Bupati Aceh Barat               Daerah Sumber  
 Bupati Aceh Barat.1  Daftar Bupati Aceh Barat  ,
 Bupati Aceh Barat Daya    NaN  PetahanaAkmal Ibrahimsejak 13 Agustus 2017  \
 Bupati Aceh Barat Daya.1  NaN  PetahanaAkmal Ibrahimsejak 13 Agustus 2017   
 
 Bupati Aceh Barat Daya                          Kediaman         Masa jabatan  \
 Bupati Aceh Barat Daya.1  Pendopo Bupati Aceh Barat Daya  5 tahun (definitif)   
 
 Bupati Aceh Barat Daya   Dibentuk Pejabat pertama               Situs web  \
 Bupati Aceh Barat Daya.1     2006      Baharud